### Bror Torstensson - bror23000

**Netflix data anlyysi**

- Datana käytössä netflix full dataset, joka sisältää palvelussa olleita tv-sarjoja sekä elokuvia. Datasta löytyy lisäksi arviointi palvelun IMDB:n tietoja. 
- Analyysissä keskitytään elokuva osioihin, sekä IMBD arvointeihin. 

In [1]:
#Alustukset

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D

data_path = "./data/full_netflix_dataset.csv"

# Datasetin osoite: https://www.kaggle.com/datasets/octopusteam/full-netflix-dataset

### Datan esikäsittely

In [2]:
#luodaan tiedostosta dataframe ja tulostetaan se.

netflix_df = pd.read_csv(data_path)

netflix_df

,title,type,genres,releaseYear,imdbId,imdbAverageRating,imdbNumVotes,availableCountries
0,The Fifth Element,movie,"Action, Adventure, Sci-Fi",1997.0,tt0119116,7.6,517695.0,"AT, CH, DE"
1,Kill Bill: Vol. 1,movie,"Action, Crime, Thriller",2003.0,tt0266697,8.2,1223453.0,"AE, AL, AO, AT, AU, AZ, BG, BH, BY, CA, CI, CM..."
2,Jarhead,movie,"Biography, Drama, War",2005.0,tt0418763,7.0,211879.0,"AD, AE, AG, AL, AO, AT, AZ, BA, BG, BH, BM, BR..."
3,Unforgiven,movie,"Drama, Western",1992.0,tt0105695,8.2,444499.0,"AU, BA, BG, CZ, HR, HU, MD, ME, MK, NZ, PL, RO..."
4,Eternal Sunshine of the Spotless Mind,movie,"Drama, Romance, Sci-Fi",2004.0,tt0338013,8.3,1106519.0,"AD, AE, AG, AL, AO, AR, AU, AZ, BA, BB, BE, BG..."
...,...,...,...,...,...,...,...,...
20191,NaN,tv,NaN,2024.0,NaN,NaN,NaN,"AL, AZ, BA, BG, BY, CY, EE, FR, GF, HU, IT, LT..."
20192,NaN,tv,Drama,2024.0,NaN,NaN,NaN,"AO, CI, CM, CV, GH, GQ, KE, MG, ML, MU, MZ, NE..."
20193,NaN,tv,"Animation, Action & Adventure",2016.0,NaN,NaN,NaN,"FJ, HK, ID, IN, JP, KR, MU, MY, PH, PK, SG, TH..."
20194,NaN,tv,NaN,NaN,NaN,NaN,NaN,"FJ, HK, ID, IN, JP, KR, MU, MY, PH, PK, SG, TH..."


Valitse datasetti ja tutustu siihen. Voit esimerkiksi tarkastella satunnaista näytettä datasta sekä rivien ja sarakkeiden määrää.

In [3]:
print(netflix_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20196 entries, 0 to 20195
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   title               19682 non-null  object 
 1   type                20196 non-null  object 
 2   genres              19928 non-null  object 
 3   releaseYear         20181 non-null  float64
 4   imdbId              18887 non-null  object 
 5   imdbAverageRating   18734 non-null  float64
 6   imdbNumVotes        18734 non-null  float64
 7   availableCountries  20196 non-null  object 
dtypes: float64(3), object(5)
memory usage: 1.2+ MB
None


Tarkista, sisältääkö datasetti puuttuvia arvoja, ja laske puuttuvan datan osuus. Päätä kontekstin perusteella, mitä tehdä puuttuville arvoille, ja toteuta tarvittavat toimenpiteet.
Tunnista ja poista datasetistä päällekkäiset arvot.

In [4]:
# Tarkistetaan puuttuvat arvot koko datasetissä
missing_values_all = netflix_df.isnull().sum()
missing_percentage_all = (missing_values_all / len(netflix_df)) * 100

print("Puuttuvat arvot ja niiden osuus:")
print(missing_values_all)
print(missing_percentage_all)

# Täytetään puuttuvat arvot numeerisissa sarakkeissa keskiarvolla
netflix_df.fillna(netflix_df.mean(), inplace=True)

# Poistetaan rivit, joissa on edelleen puuttuvaa dataa (esim. nimelliset attribuutit)
netflix_df.dropna(subset=['title', 'genres'], inplace=True)

# Tarkistetaan puuttuvat arvot uudestaan
missing_values_after_all = netflix_df.isnull().sum()
missing_percentage_after_all = (missing_values_after_all / len(netflix_df)) * 100

print("Puuttuvat arvot käsitelty:")
print(missing_values_after_all)
print(missing_percentage_after_all)

Puuttuvat arvot ja niiden osuus:
title                  514
type                     0
genres                 268
releaseYear             15
imdbId                1309
imdbAverageRating     1462
imdbNumVotes          1462
availableCountries       0
dtype: int64
title                 2.545058
type                  0.000000
genres                1.326995
releaseYear           0.074272
imdbId                6.481481
imdbAverageRating     7.239057
imdbNumVotes          7.239057
availableCountries    0.000000
dtype: float64


TypeError: can only concatenate str (not "int") to str

Jaa yksi datasetin jatkuvaa numeerista dataa sisältävä sarake sopiviin luokkiin (käytä esimerkiksi cut- tai qcut-funktiota).

Tunnista datasetistä poikkeavat arvot (outlierit). Jos datasetissä ei ole poikkeavia arvoja, voit satunnaisesti muokata osaa datasta.

Päätä, mitä tehdä poikkeaville arvoille.